In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import urllib3
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.common.action_chains import ActionChains
import requests
from urllib import request
import sys
from selenium import webdriver
from urllib.request import urlopen
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path

data = {'Country name':['Netherlands'],'url':['https://edition.cnn.com/search?q=Netherlands&from=0&size=10&page=1&sort=newest&types=all&section=']}
df = pd.DataFrame(data)

for index, row in df.iloc[:].iterrows():
    sys.path.insert(0,'D:\chromedriver\chromedriver-win64\chromedriver.exe')
    chrome_options = webdriver.ChromeOptions()
    #chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    country_name = row['Country name']
    print(country_name)
    url = row['url']
    print(url)
    website = 'https://www.reuters.com'

    if not pd.isna(url):
      driver.get(url)
      links = []
      wait = WebDriverWait(driver, 20)
      try:
        WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#onetrust-accept-btn-handler"))).click()
      except:
        pass
    
    locals()["df" + str(country_name)] = pd.DataFrame(columns = ['Date', 'Headline', 'Text'])
    start = 0
    while True:
        if start:
            try:
                element = driver.find_element(By.CSS_SELECTOR,'#search > div.search__right > div > div.search__results-pagi > div > div.pagination-arrow.pagination-arrow-right.search__pagination-link.text-active')
                sm = driver.execute_script("arguments[0].scrollIntoView({block:'center'});", element)
                element.click()
                time.sleep(1)
            except:
                break
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        data1 = soup.find('div', class_="search__results")
        start = 1
        for result in data1.find_all('a', attrs={'class':'container__link container_list-with-images__link'}):
            link = result.get('href')
            try:
                if 'videos' in link or 'gallery' in link or 'football' in link:
                    continue
                headline = result.find('span', attrs = {'data-editable':"headline"}).string
                date = result.find('div',attrs = {'class':"container__date container_list-with-images__date inline-placeholder"}).string.strip()
                text = result.find('div',attrs = {'data-editable':"description"})['data-original-text']
                locals()["df2" + str(country_name)] = pd.DataFrame({"Date":[date], "Headline":[headline], "Text":[text]})
                locals()["df" + str(country_name)] = pd.concat([locals()["df" + str(country_name)],locals()["df2" + str(country_name)]])
            except:
                pass
    
    file_path = Path(f'../Data/CNNArticles/{country_name}.csv')
    
    if file_path.exists():
        existing_df = pd.read_csv(file_path, index_col=0)
        combined_df = pd.concat([existing_df, locals()["df" + str(country_name)]], ignore_index=True)
    else:
        combined_df = locals()["df" + str(country_name)]
    
    combined_df.to_csv(file_path)



Netherlands
https://edition.cnn.com/search?q=Netherlands&from=0&size=10&page=1&sort=newest&types=all&section=
